## Dengue Fever Prediction
By: Jack Ross

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://github.com/JackRossProjects/Dengue-Fever-Predictive-Modeling/tree/master/DengueFeverData/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'